# This itereation is done. But...
I'm happy with the state of this note book now. I have tried the feature engineering and expanding windows to utilize the simple linear regression model. I have benn improving the score from 3+ to 1.9 with only the simple model. Still, I'm not done with it. I think I began to make a better sense of what I'm doing and the Time Series. Therefore, I will do the 3rd attempt on the other [notebook.](https://www.kaggle.com/thanakr/beginner-s-project-3-pipeline-and-models/edit) This time, instead of using only linear regression, I will focus on making pipeline, feature selection and more complicated models.

# Intro

Two weeks ago, I tried to learn time series prediction with this dataset with [this](https://www.kaggle.com/thanakr/beginner-s-first-project-store-sales) notebook. I tried to beat the benchmark score from the Kaggle Course of **0.5109**, but, unfortunately, the result was far from beating the score. So, here I am again. This time, in order to beat the benchmark, I need to beat my previous score of 3.44178 first. 

The score of 3.44178 is so bad. I believe it's lies somewhere in the bottom 10% of the leaderboard. Just in case if someone wondoer how I got such a score, the process is  [here.](https://www.kaggle.com/thanakr/beginner-s-first-project-store-sales)

Thankyou anyone who is reading this. All the comments or suggesttion in the discussion are highly appreciated.

Ps. I won't do the EDA with detailed interpretation here since they are already in my [first](https://www.kaggle.com/thanakr/beginner-s-first-project-store-sales) attempt. 


## Problems

Considering the calculation of RSMLE and the [math](https://medium.com/analytics-vidhya/root-mean-square-log-error-rmse-vs-rmlse-935c6cc1802a) behind, the RSMLE nullify some outliers effect by taking a log to the errors and 3.4478 is a big error here.

$$\sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2}$$

## The plan

for this attempt, I would stick with just linear regression just like the firt attempt just to see the effect of the other adjustments.

The plan for this notebook is actually the modification of some process from my last attempt:

### Sales and Transaction
* Try using MA instead of direct lag features (lags lead to a lot of na-value)
* Scale other norminal value such as onpromotion store total sales or transaction number
* School Supply Season indicator

### Holiday Data
* Using only National holiday as an holiday indicator since the sales is highly correlated to the non working day and National Holiday,unlike local holiday, is almost always a non-working day.
* Day of week dummy variable or, maybe using is_weekend feature.

### Model training
* Using the sliding windows method to train the model (later)
* Some store don't sell some product family: set the prediction to zero

### Process Control
* Use pipeline to process the inputs for the efficiency of further adjustments

I'm still not sure how much, if at all, I could improve the score but trying doesn't hurt.

Let's do it.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from warnings import simplefilter
simplefilter("ignore")  # ignore warnings to clean up output cells

import gc # for garbage cleaning

%config Completer.use_jedi = False # for autocomplete 

# Data Reading

In [ ]:
holiday_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv",)
train_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv",)
test_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
transaction_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")
sample_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv")

[](http://)# Data Formatting - Feature Engineering

### the sales and transaction data 

Task:
* Try both MA and direct lag features
* Scale other norminal value such as onpromotion store total sales or transaction number
* ~~School Supply Season indicator~~ (Tried and I don't see any big changes. Perhaps the MA of sales could serves the same purpose.

In [ ]:
train_data.tail()

In [ ]:
transaction_data.head()

In [ ]:
test_data.head()

In [ ]:
stack_sales = pd.concat([train_data,test_data])
stack_sales['date'] = pd.to_datetime(stack_sales['date'])
#stack train and test chronologically with NaN on test 'sales'

In [ ]:
stack_sales

Join Transaction data into stack_sales.

In [ ]:
transaction_data.head()
transaction_data['date'] = pd.to_datetime(transaction_data['date'])

In [ ]:
transaction_data.head()

In [ ]:
stack_sales = stack_sales.merge(transaction_data,how='left',on=['date','store_nbr'])

In [ ]:
#fill the space of no transaction and no sales to zero
stack_sales['transactions'] = stack_sales['transactions'].fillna(0)
stack_sales['sales'] = stack_sales['sales'].fillna(0)


In [ ]:
stack_sales.isna().sum().sum() #check the missing data

Rearrange the sales from the same shop and same family together in order to pick the right lag.

In [ ]:
stack_sales = stack_sales.sort_values(['family','store_nbr','date']).reset_index()
stack_sales

Let's scale the sales, onpromotion, and transactions using minmax.

In [ ]:
from sklearn.preprocessing import StandardScaler

def sd_scaling(df, column_names):
    sdScale = StandardScaler()
    for col in column_names:
        sdScale.fit(df[[col]])
        df['{}_scaled'.format(col)] = sdScale.transform(df[[col]]) 

In [ ]:
sd_scaling(stack_sales, ['sales','onpromotion', 'transactions'])

In [ ]:
stack_sales = stack_sales.drop('index', axis = 1)
stack_sales.tail()

Now let's do the Lags and Exponengtial Moving Average for sales and Transactions. The reason each lags are used can be found in the EDA section of my [first notebook](https://www.kaggle.com/thanakr/beginner-s-first-project-store-sales)

* For Sales Lag, I will use: [1 ,2 ,3 ,4 ,5 ,6 ,7 ,8 ,13 ,14] lags
* For Transactions Lag, I will use [21, 22 , 28] lags
* For MA, I'm gonna use [7, 14, 30] just to represent the average weekly, bi-weekly, monthly, quarterly, and annually trend respectively.

Let's slice the main df in to many by the shop and product family so the lag would come from it's own product family only.

In [ ]:
#find the row index of each store each proruct family so I can slice the big df at the right spot.
cut_indices = stack_sales.index[stack_sales['date'] == '2017-08-31']
cut_indices

In [ ]:
cut_dic = {}
last_cut = 0

for i,cut_index in enumerate(cut_indices):
    cut_dic[i] = stack_sales.loc[last_cut:cut_index]
    last_cut = cut_index+1    

Now we have a dic that contain separated df of a store sale for each family. I could iterate through all of them to do the sales and transaction lag correctly. (Initially the df is sorted only by store_nbr. If make lag right away it's gonna be a lag from different product family or different store.)

In [ ]:
# just cheking. Note that df contain same procuct from same store from 2013-01-01 to 2017-08-31
cut_dic[999]

Let's make lag.

In [ ]:
def make_column_lag(df, column_name, lag_list):
    for lag in lag_list:
        df['{}_lag_{}'.format(column_name, lag)] = df[column_name].shift(lag)

In [ ]:
for key in cut_dic.keys():
    make_column_lag(cut_dic[key],'sales_scaled',[1 ,2 ,3 ,4 ,5 ,6 ,7 ,8 ,13 ,14])
    make_column_lag(cut_dic[key],'transactions_scaled',[21, 22, 28])

Now calculating the Moving Average

In [ ]:
def make_column_ma(df, column_name, ma_list):
    for ma in ma_list:
        df['{}_ma_{}'.format(column_name, ma)] = df[column_name].rolling(ma, 
                                                                         min_periods = ma, 
                                                                         closed='left').mean()

# closed = "left" and ma+1 is used for not including the current day in the rolling to prevent data leak.


In [ ]:
ma_list = [8, 15, 31]
# ma_list+1 is to shift the lag further back 1 step to offset the exclusion of the current day.

for key in cut_dic.keys():
    make_column_ma(cut_dic[key],'sales_scaled',ma_list)
    make_column_ma(cut_dic[key],'transactions_scaled',ma_list)

In [ ]:
cut_dic[999].tail()

Let's stack all the cut_dic df back together.

In [ ]:
stack_sales = pd.concat(cut_dic.values())

stack_sales

In [ ]:
del cut_dic
gc.collect()

Next: drop NA, drop unnecessary columns, sort by ID to have the df back to the initial sorting.

In [ ]:
stack_sales = stack_sales.dropna()
stack_sales = stack_sales.drop(['sales_scaled','onpromotion','transactions','transactions_scaled'], axis=1)
stack_sales = stack_sales.set_index('id').sort_index()

In [ ]:
stack_sales

~~From last notebook EDA, there are some product category that are mainly sold only in some specific time. I will use binary variables representing those seasons.~~

~~First, let's see what product families need it's own season to sell.~~

I'm testing if not using this feature is ok. My assumption is that MA 7 will be quite good to act as season variable.

In [ ]:
# ### the code from the first notebook

# process_train = train_data.copy()
# process_train['date'] = pd.to_datetime(process_train['date'])
# process_train = process_train.set_index('date')
# process_train = process_train.drop('id',axis = 1)
# #process_train[['store_nbr','family']].astype('category') #to reduce ram usage

# by_fam_dic = {}
# fam_list = process_train.family.unique()

# for fam in fam_list:
#     by_fam_dic[fam] = process_train[process_train['family']==fam].sales
# fig = plt.figure(figsize=(30,50))

# for i,fam in enumerate(by_fam_dic.keys()):
#     plt.subplot(11,3,i+1)
#     plt.title('{} sale'.format(fam))
#     plt.tight_layout(pad=5)
#     sale = by_fam_dic[fam]
#     sale.plot()
#     plt.axvline(x=pd.Timestamp('2016-04-16'),color='r',linestyle='--',linewidth=2,alpha=0.3) #mark the earthquake

In [ ]:
# plt.figure().clear()
# plt.close()
# plt.cla()
# plt.clf()

In [ ]:
# del process_train
# del by_fam_dic
# del fam_list

~~Considering the charts above, I think the product families that need seasonal indicator are~~
~~* 'SCHOOL AND OFFICE SUPPLIES'~~
~~* 'FROZEN FOODS'~~
~~* 'GROCERY II'~~

~~Let's indicate the saling months of them so I can spicify the seasonal indicator. Let's add  'day of week' and 'month of year' in to the main DataFrame first.~~

In [ ]:
# stack_sales['dow'] = stack_sales['date'].dt.weekday + 1 
# stack_sales['moy'] = stack_sales['date'].dt.month

In [ ]:
# stack_sales.tail()

~~Now, let's analyze where are the periods which those products sale the most.~~

In [ ]:
# sos = stack_sales[stack_sales['family'] == 'SCHOOL AND OFFICE SUPPLIES']
# ff = stack_sales[stack_sales['family'] == 'FROZEN FOODS']
# g2 = stack_sales[stack_sales['family'] == 'GROCERY II']

In [ ]:
# def plot_product_best_month(df,title):
#     df = df.set_index(df.date)
#     sales = df.groupby('moy').sales.sum()
#     fig, ax = plt.subplots()
#     plt.plot(sales)
#     plt.title(title)
    

In [ ]:
# plot_product_best_month(sos,'school and office supply sales by month')
# plot_product_best_month(ff,'frozen foods sales by month')
# plot_product_best_month(g2,'grocery II sales by month')

# del sos
# del ff
# del g2
# gc.collect()

In [ ]:
# plt.figure().clear()
# plt.close()
# plt.cla()
# plt.clf()

~~Characteristic I saw from the sales by month are:~~
~~1. School and Office Supply sell well in April, August, September. month [4, 8, 9]~~
~~2. Frozen Foods sell well in December. month [12]~~
~~3. Grocery II sell well in May, June, July, December. month [5,6,7,12]~~

~~Let's add the columns for those seasons.~~

In [ ]:
# stack_sales[['sos_ss', 'ff_ss', 'g2_ss']] = 0
# stack_sales.loc[stack_sales['moy'].isin([4,8,9]), 'sos_ss'] = 1 
# stack_sales.loc[stack_sales['moy'].isin([12]), 'ff_ss'] = 1
# stack_sales.loc[stack_sales['moy'].isin([5,6,7,12]), 'g2_ss'] = 1 

# #stack_sales[['sos_ss', 'ff_ss', 'g2_ss']] = stack_sales[['sos_ss', 'ff_ss', 'g2_ss']].astype('category')

The planned process for the sales features are done.

### Sales and Transaction 
* **DONE** -> Try using MA and direct lag features
* **DONE** -> Scale other norminal value such as onpromotion store total sales or transaction number
~~* **DONE** -> School Supply (and other) Season indicator~~

Let's move on to process the Holiday data.
### Holiday Data
* Using only National holiday as an holiday indicator since the sales is highly correlated to the non working day
* Day of Week dummy indicator or is_weekend indicator.

In [ ]:
holiday = holiday_data.set_index('date')
holiday.index = pd.to_datetime(holiday.index)

In [ ]:
nat_holiday_data = holiday.loc[holiday['locale'].str.contains('National')]
nat_holiday_data

In [ ]:
calendar = pd.DataFrame(index = pd.date_range('2013-01-01','2017-08-31'))
calendar = calendar.join(nat_holiday_data).fillna(0)
calendar

Still need New Year mentioned since the sales in New year day is almost zero across the nation.

In [ ]:
calendar['is_ny'] = 0
calendar.loc['2013-01-01', 'is_ny'] = 1
calendar.loc['2014-01-01', 'is_ny'] = 1
calendar.loc['2015-01-01', 'is_ny'] = 1
calendar.loc['2016-01-01', 'is_ny'] = 1
calendar.loc['2017-01-01', 'is_ny'] = 1

In [ ]:
calendar['dow'] = calendar.index.weekday+1 #set Monday to 1 and Sunday to 7
calendar['moy'] = calendar.index.month 

In [ ]:
calendar.type.unique()

In [ ]:
calendar['is_non_workday'] = 0

calendar.loc[calendar['locale'] == 'National', 'is_non_workday'] = 1 #make national events a non_workday
calendar.loc[calendar['dow'] > 5 , 'is_non_workday'] = 1 #make Sat and Sun non_workday
calendar.loc[calendar['type'] == 'Work Day', 'is_non_workday'] = 0 #make Work Day holiday a working day
calendar.loc[(calendar['transferred'] == True) & 
             (calendar['dow'] < 6), 
             'is_non_workday'] = 0 #mark a transfered weekday a working day

In [ ]:
calendar

let's format football, shoppine events, the earthquake.

In [ ]:
calendar['is_football'] = 0
calendar['is_eq'] = 0
calendar['is_shopping'] = 0

#mark football matches
calendar.loc[(calendar['locale'] == 'National') & 
             (calendar['description'].str.contains('futbol')), 'is_football'] = 1

#mark the earthquake
calendar.loc[(calendar['locale'] == 'National') & 
             (calendar['description'].str.contains('Terremoto')), 'is_eq'] = 1

#mark Cyber Mondays
calendar.loc[(calendar['locale'] == 'National') & 
             (calendar['description'].str.contains('Cyber Monday')), 'is_shopping'] = 1

#mark Black Fridays
calendar.loc[(calendar['locale'] == 'National') & 
             (calendar['description'].str.contains('Black Friday')), 'is_shopping'] = 1

In [ ]:
calendar

let's format day of week to dummy variable.

In [ ]:
dow_dummy = pd.get_dummies(calendar['dow'],prefix = 'dow',drop_first = True)
dow_dummy

In [ ]:
moy_dummy = pd.get_dummies(calendar['moy'],prefix = 'moy',drop_first = True)
moy_dummy

Merge dow_dummy to calendar and rearrange the columns.

In [ ]:
calendar = calendar.merge(dow_dummy, how='left', left_index = True, right_index = True)
calendar = calendar.merge(moy_dummy, how='left', left_index = True, right_index = True)

In [ ]:
calendar.columns #see what are the columns

In [ ]:
#pick only needed columns
calendar_rdy = calendar[['is_ny',
       'is_non_workday', 'is_football', 'is_eq', 'is_shopping',
       'dow_2', 'dow_3', 'dow_4', 'dow_5', 'dow_6', 'dow_7', 'moy_2', 'moy_3',
       'moy_4', 'moy_5', 'moy_6', 'moy_7', 'moy_8', 'moy_9', 'moy_10',
       'moy_11', 'moy_12']]

#calendar_rdy[['is_ny','is_non_workday', 'is_football', 'is_eq', 'is_shopping', 'dow_2','dow_3', 'dow_4', 'dow_5', 'dow_6', 'dow_7']] = calendar_rdy[['is_ny','is_non_workday', 'is_football', 'is_eq', 'is_shopping', 'dow_2','dow_3', 'dow_4', 'dow_5', 'dow_6', 'dow_7']].astype('category')

In [ ]:
del calendar
del nat_holiday_data
del holiday
gc.collect()

In [ ]:
calendar_rdy.tail()

### Holiday Data
* **DONE ->** Using only National holiday as an holiday indicator since the sales is highly correlated to the non working day
* **DONE ->** Day of Week dummy indicator

Now, let's merge the calenda to the stack_sales.

In [ ]:
stack_sales = stack_sales.merge(calendar_rdy, how ='left', left_on = 'date', right_on = calendar_rdy.index)

In [ ]:
pd.set_option('display.max_columns', None)
stack_sales.tail()

Store_nbr and product family should be dummy too.

In [ ]:
nbr_dummy = pd.get_dummies(stack_sales['store_nbr'],prefix='store',drop_first=True)
nbr_dummy

In [ ]:
family_dummy = pd.get_dummies(stack_sales['family'],prefix='fam',drop_first=True)
family_dummy

Merge the Dummy back

In [ ]:
stack_sales.drop(['store_nbr','family'],axis = 1, inplace = True)

In [ ]:
stack_sales = stack_sales.merge(nbr_dummy, how ='left', left_index = True, right_index = True)
stack_sales = stack_sales.merge(family_dummy, how ='left', left_index = True, right_index = True)

In [ ]:
del calendar_rdy
del nbr_dummy
del dow_dummy
del family_dummy
gc.collect()

In [ ]:
stack_sales.isna().sum().sum()

In [ ]:
for i in stack_sales.columns:
    print("'"+i+"'"+",")

In [ ]:
category_col = ['is_ny','is_non_workday','is_football','is_eq','is_shopping','dow_2','dow_3','dow_4','dow_5','dow_6','dow_7','moy_2','moy_3',
                   'moy_4','moy_5','moy_6','moy_7','moy_8','moy_9','moy_10','moy_11','moy_12','store_2','store_3','store_4','store_5','store_6',
                   'store_7','store_8','store_9','store_10','store_11','store_12','store_13','store_14','store_15','store_16','store_17','store_18','store_19',
                   'store_20','store_21','store_22','store_23','store_24','store_25','store_26','store_27','store_28','store_29','store_30','store_31',
                   'store_32','store_33','store_34','store_35','store_36','store_37','store_38','store_39','store_40','store_41','store_42','store_43',
                   'store_44','store_45','store_46','store_47','store_48','store_49','store_50','store_51','store_52','store_53','store_54','fam_BABY CARE',
                   'fam_BEAUTY','fam_BEVERAGES','fam_BOOKS','fam_BREAD/BAKERY','fam_CELEBRATION','fam_CLEANING','fam_DAIRY','fam_DELI','fam_EGGS','fam_FROZEN FOODS',
                   'fam_GROCERY I','fam_GROCERY II','fam_HARDWARE','fam_HOME AND KITCHEN I','fam_HOME AND KITCHEN II','fam_HOME APPLIANCES','fam_HOME CARE',
                   'fam_LADIESWEAR','fam_LAWN AND GARDEN','fam_LINGERIE','fam_LIQUOR,WINE,BEER','fam_MAGAZINES','fam_MEATS','fam_PERSONAL CARE','fam_PET SUPPLIES',
                   'fam_PLAYERS AND ELECTRONICS','fam_POULTRY','fam_PREPARED FOODS','fam_PRODUCE','fam_SCHOOL AND OFFICE SUPPLIES','fam_SEAFOOD',]

stack_sales[category_col] = stack_sales[category_col].astype('category')

In [ ]:
stack_sales = stack_sales.reset_index()
stack_sales.rename(columns={'index':'id'} ,inplace=True)

In [ ]:
stack_sales.set_index('date', inplace=True)

In [ ]:
stack_sales

In [ ]:
stack_sales.info(verbose=True)

Let's check the collinearity for each variables.

In [ ]:
corr = stack_sales.corr()

In [ ]:
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
del corr
gc.collect()

The correlation heatmap show that there are so many highly linearily corealated inputs. Therefore, I will drop some of them.

What I get from the correlation DF is these:
* Transaction lag and Transaction MA are highy correlated -> use only one -> Use only MA 15 (the middle ground)
* Sales lag and Sales MA are highy correlated -> use only one -> Use only MA 15 (the midding ground)


In [ ]:
stack_sales = stack_sales.drop(['sales_scaled_lag_1',
                                'sales_scaled_lag_2',
                                'sales_scaled_lag_3',
                                'sales_scaled_lag_4',
                                'sales_scaled_lag_5',
                                'sales_scaled_lag_6',
                                'sales_scaled_lag_7',
                                'sales_scaled_lag_8',
                                'sales_scaled_lag_13',
                                'sales_scaled_lag_14',
                                'transactions_scaled_lag_21',
                                'transactions_scaled_lag_22',
                                'transactions_scaled_lag_28',
                                'sales_scaled_ma_8',
                                'sales_scaled_ma_31',
                                'transactions_scaled_ma_8',
                                'transactions_scaled_ma_31'], axis=1)

Let's check the correlation again.

In [ ]:
corr = stack_sales.corr()

In [ ]:
corr.style.background_gradient(cmap='winter')

In [ ]:
stack_sales

Looks better. Let's continue.

# Model training
Let's construct the model. Again, I'm still using the normal linear regression here just to compare the score to the last time. 

### Tasks
* Using the sliding windows method to train the model (later)
* Some store don't sell some product family: set the prediction to zero

First, let's take a note which store doen't sell which items.

In [ ]:
zeros = train_data[['store_nbr','family','sales']].groupby(['store_nbr','family']).sum()

In [ ]:
zeros = zeros[zeros['sales']==0].reset_index()

In [ ]:
zeros = zeros[['store_nbr','family']].set_index('store_nbr')

In [ ]:
zeros.head()

In [ ]:
zeros = zeros.groupby('family').groups

In [ ]:
zeros

Now I have a dic that has not available product families as keys and shop as the element. Let's specify the id for set zero the sales at the end of the process.

In [ ]:
id=[]
for family in zeros.keys():
    id_to_extend = test_data.loc[(test_data['family'] == family) & 
                                 (test_data['store_nbr'].isin(zeros[family])), 'id'].values
    id.extend(id_to_extend)

In [ ]:
id[:10] #just checking

The list of zero sales is ready. This will be used at the end of the prediction.

In [ ]:
from sklearn.linear_model import LinearRegression


# X = stack_sales.loc['2013-01-01':'2017-08-15'].drop(['id','sales'],axis=1)
# y = stack_sales.loc['2013-01-01':'2017-08-15', 'sales']
X_test = stack_sales.loc['2017-08-16':].drop(['id','sales'],axis=1)


ln = LinearRegression(fit_intercept=False) 
#the data has been scaled with standardscaler and has mean of zero. The fit intercept is unneccessary.

Fit the model

### Now, time for the Sliding Window model training

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
slide = TimeSeriesSplit(n_splits=5,test_size =119900) #119900 is actually 20% of the train dataaset

In [ ]:
X_for_split = stack_sales.loc[:'2017-08-16']

In [ ]:
X_for_split

In [ ]:
split_id_list = list(slide.split(X_for_split))
# split_id = slide.split(X_for_split)

In [ ]:
split_id_list

In [ ]:
X_try_id = []
X_val_try_id = []


for i in range(len(split_id_list)):
      X_try_id.append(split_id_list[i][0])
      X_val_try_id.append(split_id_list[i][1])

In [ ]:
X_try_id

In [ ]:
X_val_try_id

In [ ]:
X_slide = X_for_split.set_index('id').loc[X_try_id[i]]
X_slide

In [ ]:
y_slide = X_for_split.set_index('id').loc[X_try_id[i], 'sales']
y_slide

In [ ]:
from sklearn.metrics import mean_squared_log_error

for i in range(len(X_try_id)):
    ln.fit(X_slide.drop('sales',axis=1), y_slide)
    y_pred = ln.predict(X_for_split.set_index('id').loc[X_val_try_id[i]].drop('sales', axis=1))
    y_val = X_for_split.set_index('id').loc[X_val_try_id[i], 'sales']
    rsmle = mean_squared_log_error(abs(y_val),abs(y_pred),squared=False)
    print('Run# {} : RSMLE = {}'.format(i+1,rsmle))

RSMLE is getting better at each epoch. Let's prdict the real test_data and submit for the result.

In [ ]:
y_pred_slide = ln.predict(X_test)
y_pred_slide

Let's submit the output.

In [ ]:
test_data['sales'] = y_pred_slide

Set zero the zeros

In [ ]:
test_data.loc[test_data['id'].isin(id), 'sales'] = 0
test_data.loc[test_data['sales'] < 0, 'sales'] = 0 #set zero for negative sales.
test_data[test_data['sales'] < 0] #check to confirm no negative sales

In [ ]:
test_data[['id','sales']].to_csv('submission.csv', index = False) # Submit

* ### Score Log
* #### 1st. edit -> RSMLE 3.44178
    * LinearRegression 
    * without checking collinearity 
    * no zero sales formatting
* #### 2nd. edit -> RSMLE 2.37238
    * LinearRegression
    * **Flawed data formatting**
        * Need to drop NA
        * Wrong formatted data lag
    * Collinearity  Checked
    * Manually mark the zero sales product
* #### 3rd. edit -> RSMLE 1.99939 ***The score Improve Significantly***
    * LinearRegression
    * Drop NA / Lag fixed
        * Reduced sales / transactions features to only MA15
    * Collinearity  Checked
    * Manually mark the zero sales product
    
* #### 4th. edit -> RSMLE 1.97205 ***Best Score Yet***
    * LinearRegression + Expanding windows
    * Drop NA / Lag fixed
        * Reduced sales / transactions features to only MA15
    * Collinearity  Checked
    * Manually mark the zero sales product
    

# Problems and findings notes

1. I have learn that the feature's collinearity is the main reason for the abysmal result in the first iteration. This is prone to happen when using many dummy variables. So, keep an eye out for them.
2. I have been formatting the input incorectly.
    * Include the current sales into the rolling MA sales/transaction windows. This is data leakage
    * Make the lag with pd.DF.shift() without sorting the procuct and the store. This will case the lag to come from different store or product family.
    * Didn't drop NA, instead, I just fill with all the na with zeros, which, corrupted the dataset.